# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [4]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 619


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [7]:
# Set the API base URL
url = "https://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"{url}q={city}&appid={weather_api_key}&units=metric"

    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        response = requests.get(city_url)
        city_weather = response.json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_date = city_weather['dt']

        # Append the City information into city_data list
        city_data.append({"City": city,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except KeyError:
        print("City not found. Skipping...")
        pass

    # pause to avoid rate limiting
    time.sleep(1)

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | edinburgh of the seven seas
Processing Record 2 of Set 1 | grytviken
Processing Record 3 of Set 1 | coronel dorrego
Processing Record 4 of Set 1 | suva
Processing Record 5 of Set 1 | galle
Processing Record 6 of Set 1 | crane
Processing Record 7 of Set 1 | chonchi
Processing Record 8 of Set 1 | college
Processing Record 9 of Set 1 | erdenet
Processing Record 10 of Set 1 | magadan
Processing Record 11 of Set 1 | nar'yan-mar
Processing Record 12 of Set 1 | sydney mines
Processing Record 13 of Set 1 | nadym
Processing Record 14 of Set 1 | sao jose da coroa grande
Processing Record 15 of Set 1 | taiohae
City not found. Skipping...
Processing Record 16 of Set 1 | cabo san lucas
Processing Record 17 of Set 1 | kodiak
Processing Record 18 of Set 1 | amfissa
Processing Record 19 of Set 1 | freeport
Processing Record 20 of Set 1 | minas de marcona
Processing Record 21 of Set 1 | dejen
Processing Record 22

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data) 

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
plt.scatter(city_data_df['Lat'], city_data_df['Max Temp'], alpha=0.75, linewidths=1, edgecolors='black')

# Incorporate the other graph properties
plt.title('City Max Latitude vs. Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (c)')
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
plt.scatter(city_data_df['Lat'], city_data_df['Humidity'], alpha=0.75, linewidths=1, edgecolors='black')

# Incorporate the other graph properties
plt.title('City Latitude vs. Humidity')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.scatter(city_data_df['Lat'], city_data_df['Cloudiness'], alpha=0.75, linewidths=1, edgecolors='black')

# Incorporate the other graph properties
plt.title('City Latitude vs. Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.scatter(city_data_df['Lat'], city_data_df['Wind Speed'], alpha=0.75, linewidths=1, edgecolors='black')

# Incorporate the other graph properties
plt.title('City Latitude vs. Wind Speed')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (m/s)')
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
def plot_linear_regression(x, y, x_label, y_label, title):
    (slope, intercept, r_value, p_value, std_err) = stats.linregress(x, y)

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df[city_data_df['Lat'] >= 0]

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df[city_data_df['Lat'] < 0]

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Max Temp']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(northern_hemi_df['Lat'], northern_hemi_df['Max Temp'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")

plt.xlabel('Latitude')
plt.ylabel('Max Temp')
plt.title("Northern Hempisphere: Latitude vs. Max Temp")


In [ ]:
# Linear regression on Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Max Temp']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(southern_hemi_df['Lat'], southern_hemi_df['Max Temp'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")
plt.xlabel('Latitude')
plt.ylabel('Max Temp')
plt.title("Southern Hempisphere: Latitude vs. Max Temp")


**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Humidity']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(northern_hemi_df['Lat'], northern_hemi_df['Humidity'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")

plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.title("Northern Hempisphere: Latitude vs. Humidity")


In [ ]:
# Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Humidity']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(southern_hemi_df['Lat'], southern_hemi_df['Humidity'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.title("Southern Hempisphere: Latitude vs. Humidity")

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Cloudiness']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(northern_hemi_df['Lat'], northern_hemi_df['Cloudiness'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")

plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.title("Northern Hempisphere: Latitude vs. Cloudiness")

In [ ]:
# Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Cloudiness']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(southern_hemi_df['Lat'], southern_hemi_df['Cloudiness'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.title("Southern Hempisphere: Latitude vs. Cloudiness")

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Wind Speed']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(northern_hemi_df['Lat'], northern_hemi_df['Wind Speed'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")

plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.title("Northern Hempisphere: Latitude vs. Wind Speed")

In [ ]:
# Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Wind Speed']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regression_values = slope * x_values + intercept 

plt.plot(x_values, regression_values, color ='red')
plt.scatter(southern_hemi_df['Lat'], southern_hemi_df['Wind Speed'], linewidths=1, alpha=0.75)

print(f"The r^2-value is: {rvalue**2}")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.title("Southern Hempisphere: Latitude vs. Wind Speed")

**Discussion about the linear relationship:** YOUR RESPONSE HERE